In [ ]:
%%bash
pip install flair
pip install nltk

In [ ]:
import os
from collections import defaultdict

from flair.data import Sentence
from flair.models import SequenceTagger
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Change the path to where the parallel data file is located on your device/drive
DATA_PATH = "drive/MyDrive/Data/parallel/Tanzil-en-ha.txt"

UNK = "<unk>"

In [ ]:
# load tagger
tagger = SequenceTagger.load("flair/upos-english")

In [ ]:
with open(DATA_PATH) as f:
    counter = 0
    line_dict = defaultdict(list)
    tag_dict = dict()
    for line in f:
      if counter%1000 == 0:
        print(counter)
      # Split line at delimiter
      line = line.split("|||")
      # Only look at line with the right format.
      if len(line) == 2:
        src, _ = line
        words = src.split()
        tags = [(UNK, 0) for i in range(len(words))]
        sentence = Sentence(src, use_tokenizer=False)  
        # Tag words
        if src in tag_dict:
          tags = tag_dict[src]
        else:
          tagger.predict(sentence)
          tags = [(tags["value"], tags["confidence"]) for tags in sentence.to_dict()["all labels"]]
        line_dict[counter] = tags
        tag_dict[src] = tags
        counter += 1

Finally, the results are written to the given output path.

In [ ]:
OUT = "OUT/PATH"

with open(OUT, "w", encoding="utf-8") as file:
  for i in range(len(line_dict)):
    tags = line_dict[i]
    tag_str = ["{}-{}".format(tag, score) for tag, score in tags]
    file.write("{}\n".format(" ".join(tag_str)))